In [61]:
os.chdir('Analysis/')

In [234]:
from __future__ import division, print_function
import os, glob, re
import numpy
from math import log #, log10
import json
import pyfits
from collections import OrderedDict

from analysis import Analyzer

In [292]:
pNames = my_decorator(parNames)

In [293]:
pNames(['grbm'])

['alpha', 'beta', 'tem', 'norm']
['alpha', 'beta', 'tem', 'norm']


In [275]:
mods = ['sbpl', 'grbm']
verz = ['-01-','-01-']
methods = ['BXA','BXA']
burst = 'bn080916009'
det = 'G'
model = mods
version = verz
method = methods
params =  my_decorator(parNames)

In [276]:
my_decorator(parNames, getValues(burst, det, model, version, method, params))

<function wrapper at 0x105907320>
Setting bxa_dir to: 
 /Users/KimiZ/GRBs2/analysis/LAT/bn080916009/BXA/GBM/


AttributeError: 'list' object has no attribute 'startswith'

In [257]:
pNames(['sbpl', 'grbm'])

['alpha', 'beta', 'ebreak', 'norm']
['alpha', 'beta', 'tem', 'norm']


In [157]:
Zoldak = my_decorator(getValues)

In [ ]:
modZ = my_decorator(getValues)

In [158]:
kz2(burst='bn080916009', det='G', 
    modName=['grbm','sbpl'], 
    version=['-01-','-01-'], 
    method=['BXA','BXA'], 
    paramNames= kz(['grbm', 'sbpl']))

['alpha', 'beta', 'tem', 'norm']
['alpha', 'beta', 'ebreak', 'norm']


TypeError: wrapper() got an unexpected keyword argument 'burst'

In [486]:
p = getValues(burst='bn080916009', det='L', 
    modName='grbm', 
    version='-01-',
    method='BXA', 
    paramNames= mm)

['alpha', 'beta', 'tem', 'norm']
Setting bxa_dir to: 
 /Users/KimiZ/GRBs2/analysis/LAT/bn080916009/BXA/GBMwLAT/
/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/BXA/GBMwLAT/grbm/bxa_fitresults_grbm_-01-_L_.fit
OrderedDict([('alpha__1', [-1.0261301941989505, -1.0504917692117177, -1.0108496177344828]), ('beta__2', [-2.1961747832013394, -2.2118715235305162, -2.187204678512312]), ('tem__3', [519.22663537909375, 486.94436728067308, 579.5837754287212]), ('norm__4', [0.017587351222993219, 0.017034773315515072, 0.017956890814037643])])


In [487]:
b = getValues(burst='bn080916009', det='L', 
    modName='sbpl', 
    version='-01-',
    method='BXA', 
    paramNames= mm)

['alpha', 'beta', 'tem', 'norm']
Setting bxa_dir to: 
 /Users/KimiZ/GRBs2/analysis/LAT/bn080916009/BXA/GBMwLAT/
/Users/KimiZ/GRBs2/analysis/LAT/bn080916009/BXA/GBMwLAT/sbpl/bxa_fitresults_sbpl_-01-_L_.fit
OrderedDict([('alpha__1', [-1.1093541084556167, -1.1231789991061667, -1.0925475070288904]), ('beta__2', [-2.1913561811028215, -2.2026726344413343, -2.1815726604756018]), ('tem__3', [270.86186477877789, 253.90687283818764, 287.10019070829043]), ('norm__4', [0.014589863969694064, 0.014404443141418422, 0.014766127627113575])])


In [481]:
class Parameter(object):
    def __init__(self, burst=None, det=None, model=None, version=None, method=None):
        # burst and det are the only two I want to send over to Analyzer, 
        # so the rest need to be None.  If I want to send them over, remove =None.
        #Analyzer.__init__(self, burst=None, det=None, model=None, version=None, method=None)
        self._burst     = burst
        self._det       = det
        self._modName   = model
        self._version   = version
        self._method    = method
        #self.comparesults = self.CompareResults()

    @property
    def parNames(self):
        '''
        Given a model name, it will return the parameter names of the model.
        See file 'ParameterNames.json in the $PYX/Analysis/ directory.
        '
        '''
        filename = "/Users/KimiZ/GRBs2/analysis/pyxspec_programs/Analysis/ParameterNames.json"
        f                = open(filename, "r")
        ParamDict        = json.load(open(filename, 'r'), encoding='utf-8')
        self._parNames   = ParamDict[self._modName]
        self._parNames   = [str(i) for i in self._parNames]
        return self._parNames

        
    def getValues(self, method=None):
        if self._method:
            self._method = self._method
        else:
            self._method = method
        #self._method = method
        if 'BXA' in self._method:
            self._mType = 'bxa'
            idx = 1   # bxa and pyxspec have different number of tables in their files.
        elif 'PYX' in self._method:
            self._mType = 'xspec'
            idx = 2

        analyze  = Analyzer(self._burst, self._det, self._modName, method=self._method)
        direc    = analyze.ModelDir()
        filename = os.path.join(direc, "%s_fitresults_%s_%s_%s_.fit"
                                %(self._mType, self._modName, self._version, self._det))
        _data = pyfits.open(filename)
        
        new_data = OrderedDict()
        for n,par in enumerate(self._parNames):
            n=n+1
            name       = "PARAM%i"%n
            if ('norm' in par and 'BXA' in self._method):
                Val   = 10**(_data[idx].data[name][0][0])  # param values
                errL  = 10**(_data[idx].data[name][0][1])  # 1sigma errors Lower
                errU  = 10**(_data[idx].data[name][0][2])  # 1sigma errors Upper
            else:  
                Val        = _data[idx].data[name][0][0]  # param values
                errL       = _data[idx].data[name][0][1]  # 1sigma errors Lower
                errU       = _data[idx].data[name][0][2]  # 1sigma errors Upper
            new_data[par+"__%i"%n] = [Val, errL, errU]
        return new_data

In [477]:
class CompareResults(object):
    def __init__(self):
        self.name = "X"
        
    def ErrorSpread(self, *args):
        kim = Parameter(*args)
        return kim._getValues()
    
    def ParamSpread(self):
        pass


    def ZZZ(self):
        #super(CompareResults, self).__setattr__()
        print(self._burst)

        

In [478]:
me = CompareResults()

In [480]:
me.ErrorSpread(burst='bn080916009', det='L', 
               model='grbm', version='-01-', 
               method='BXA')

TypeError: ErrorSpread() got an unexpected keyword argument 'burst'

In [464]:
kz = Parameter(burst='bn080916009', det='L', 
               model='grbm', version='-01-', 
               method='BXA')

In [465]:
kz.parNames

['alpha', 'beta', 'tem', 'norm']

In [466]:
kz._getValues()

Setting bxa_dir to: 
 /Users/KimiZ/GRBs2/analysis/LAT/bn080916009/BXA/GBMwLAT/


OrderedDict([('alpha__1',
              [-1.0261301941989505, -1.0504917692117177, -1.0108496177344828]),
             ('beta__2',
              [-2.1961747832013394, -2.2118715235305162, -2.187204678512312]),
             ('tem__3',
              [519.22663537909375, 486.94436728067308, 579.5837754287212]),
             ('norm__4',
              [0.017587351222993219,
               0.017034773315515072,
               0.017956890814037643])])

In [439]:
kz = Parameter(burst='bn080916009', det='L', 
               model='grbm', version='-01-', 
               method='BXA')

In [440]:
kz.parNames

['alpha', 'beta', 'tem', 'norm']

In [442]:
kz = CompareResults(burst='bn080916009', det='L')#, 
                    #model='grbm', version='-01-', 
                    #method='BXA')

TypeError: __init__() got an unexpected keyword argument 'det'

In [342]:
kz = Parameter(burst='bn080916009', det='L', 
               model=['grbm', 'sbpl'], 
               version=['-01-', '-01-'], 
               method=['BXA','BXA'])

In [343]:
kz.parNames

TypeError: unhashable type: 'list'

In [67]:
kz.parNames

If you'd like to know the parameters for another model,
use parNames('modelname')
Using model name from initilization. 


['alpha', 'beta', 'tem', 'norm']

In [52]:
kz = Parameter('grbm', 1)

In [53]:
kz.parNames

Exception: Need to enter a model name.

In [25]:
method = 'BXA'

In [26]:
parNames = ['a', 'b', 'ep', 'norm']

In [27]:
for n,par in enumerate(parNames):
    if ('norm' in par and 'BXA' in method):
        Val   = 10**(4.23)  # param values
        errL  = 10**(0.123)  # 1sigma errors Lower
        errU  = 10**(0.924)  # 1sigma errors Upper
    else:  
        Val        = 4.24  # param values
        errL       = 0.123  # 1sigma errors Lower
        errU       = 0.924  # 1sigma errors Upper
    print(par, Val, errL, errU)

('a', 4.24, 0.123, 0.924)
('b', 4.24, 0.123, 0.924)
('ep', 4.24, 0.123, 0.924)
('norm', 16982.43652461746, 1.3273944577297394, 8.394599865193975)


In [43]:
class C(object):
    def __init__(self, modelName, parNames):
        self.modelName = modelName
        self.parNames = parNames
    def getX(self, x):
        '''
        If this is called before getY is, the self.x is adopted 
        by the getY and getY will return the x we give getX.
        getX(2) will then set self.x = 2 for the ENTIRE class.
        If we don't call this before getY, then getY will throw an error.
        '''
        return x
    def getY(self, y):
        self.y = y
        print("x and y:", self.x, self.y)

In [2]:
class GetVals(object):
    def __init__(self, parName, parStrategy):
        self.name = parName
        if isinstance(parStrategy, defaultVals):
            self.__iType = 0
        else:
            self.__iType = 1 



In [6]:
class Component(object):
    def __init__(self, compName, parNames):
        self.name = compName
        

        
        for parName in parNames:
            setattr(self, parName, GetVals(parName, defaultVals))
    

In [7]:
modelName = 'grbm'
parameterNames = ['alpha','beta','epeak','norm']

In [8]:
kz = Component(modelName, parameterNames)

In [11]:
kz.defaultVals()

[-1, -2, 300.0, 0.001]

In [13]:
kz.modelVals()

[-1.435, -2.455, 602.88, 0.002482]

In [ ]:
class Account:
    def __init__(self):
        # 2. to refer to the inner class, you must use self.Bank
        # 3. no need to use an inner class here
        self.bank = self.Bank()

    class Bank:
        def __init__(self):
            self.balance = 100000

        # 4. in your original code, you had a method with the same name as 
        #    the attribute you set in the constructor. That meant that the 
        #    method was replaced with a value every time the constructor was 
        #    called. No need for a method to do a simple attribute lookup. This
        #    is Python, not Java.

        def withdraw(self, amount):
            self.balance -= amount

        def deposit(self, amount):
            self.balance += amount

In [ ]:
a = Account()

In [ ]:
print(a.bank.balance)

In [ ]:
class Account(object):
    def __init__(self):
        self.bank = self.Bank()

    class Bank(Account):
        def __init__(self):
            self.balance = 100000

        def withdraw(self, amount):
            self.balance -= amount

        def deposit(self, amount):
            self.balance += amount

In [ ]:
class Account:
    def __init__(self):
        self.bank = account.Bank()

    class Bank:
        def __init__(self):
            self.balance = 100000

        def whitdraw(self, amount):
            self.balance -= amount

        def deposit(self, amount):
            self.balance += amount

a = account()
print a.bank.balance

In [ ]:
a = Account()
print(a.bank.balance)

In [ ]:
a.bank.balance

In [ ]:
a.bank.withdraw(100)

In [ ]:
a.bank.balance